In [ ]:
#! pip install dataretrieval
#! pip install bmipy
#! pip install eto
#! pip install hydrostats
import sys 
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy.stats import kstest
import bmi_cfe_perturb_ens
import new_bmi_EnKF as bmi_EnKF
import bmi_cfe
import bmi_PET
from fao_pet import ETRCalculator

In [ ]:
#######run pet and get values
pet_instance = bmi_PET.BMI_pet_model()
pet_instance.initialize(cfg_file='./pet.json', current_time_step=1)
pet_instance.update()
pet_instance.finalize()
pet = pet_instance.get_value('etr').values
pet = pet / 3600

In [ ]:
np.sum(pet)

In [ ]:
enkf_instance = bmi_EnKF.BMIEnKF('./input_enkf_new.json')
cfe_purturb_instance_open = bmi_cfe_perturb_ens.BMI_CFE('./new_cfe_perturb.json')
cfe_purturb_instance_synth = bmi_cfe_perturb_ens.BMI_CFE('./new_cfe_perturb.json')
cfe_instance = bmi_cfe.BMI_CFE('./new_cfe.json')

In [ ]:
#initialization
cfe_purturb_instance_open.initialize()
cfe_purturb_instance_synth.initialize()
cfe_instance.initialize()
enkf_instance.initialize()

In [ ]:
### Observation data

observation_no_noise = []
state_sm_all =[]

with open(cfe_instance.forcing_file, 'r') as f:
    df_forcing = pd.read_csv(f)
    
for t, precip in enumerate(df_forcing['APCP_surface']):
    precip = precip / 1000
    cfe_instance.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
  #  cfe_instance.set_value('water_potential_evaporation_flux', pet[t])
    cfe_instance.update()
    obs = cfe_instance.get_value("land_surface_water__runoff_volume_flux")
    observation_no_noise.append(obs)
    state = cfe_instance.get_value("SOIL_CONCEPTUAL_STORAGE")
    state_sm_all.append(state)
    
cfe_instance.finalize()

perturbation_factor_obs = np.random.standard_normal(size=None)
error_factor_obs = 0.09
observation_no_noise = np.array(observation_no_noise)       
observation_noisy = (perturbation_factor_obs * observation_no_noise * error_factor_obs) + observation_no_noise




In [ ]:
######### open loop

open_loop_simulations = []
open_loop_simulations_mean = []
ostate_variables_all_mean = []

with open(cfe_purturb_instance_open.forcing_file, 'r') as f:
    df_forcing = pd.read_csv(f)



for t, precip in enumerate(df_forcing['APCP_surface']):
    precip = precip / 1000
    cfe_purturb_instance_open.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
    #cfe_purturb_instance_open.set_value('water_potential_evaporation_flux', pet[t])
    cfe_purturb_instance_open.update()
    simulations = cfe_purturb_instance_open.get_value("land_surface_water__runoff_volume_flux")
    simulations = np.array(list(simulations.values()))
    open_loop_simulations.append(simulations)
    mean_simulations = np.mean(simulations)
    open_loop_simulations_mean.append(mean_simulations)
    
    ostate_variables = cfe_purturb_instance_open.get_value("SOIL_CONCEPTUAL_STORAGE")
    ostate_variablesssss = np.array(list(ostate_variables))
    ostate_variables_mean = np.mean(ostate_variablesssss)
    ostate_variables_all_mean.append(ostate_variables_mean)

cfe_purturb_instance_open.finalize()    


In [ ]:
######Synthetic loop


with open(cfe_purturb_instance_synth.forcing_file, 'r') as f:
    df_forcing = pd.read_csv(f)


da_simulations_all = []
da_simulations_all_mean = []
state_variables_all = []
state_variables_all_mean = []
simulations_all = []
observations_all = []
updated_state_all = []
updated_simulations_all = []
gw_state_all = []
gw_state_all_mean = []
updated_gw_state_all = []


for t, precip in enumerate(df_forcing['APCP_surface']):
    precip = precip / 1000
    cfe_purturb_instance_synth.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
    cfe_purturb_instance_synth.set_value('water_potential_evaporation_flux', pet[t])
    cfe_purturb_instance_synth.update()
    simulations = cfe_purturb_instance_synth.get_value("land_surface_water__runoff_volume_flux")
    simulationsss = np.array(list(simulations.values()))
    da_simulations_all.append(simulationsss)
    mean_simulations = np.mean(simulationsss)
    da_simulations_all_mean.append(mean_simulations)
    
    state_variables = cfe_purturb_instance_synth.get_value("SOIL_CONCEPTUAL_STORAGE")
    state_variablesssss = np.array(list(state_variables))
    state_variables_all.append(state_variables)
    state_variables_mean = np.mean(state_variablesssss)
    state_variables_all_mean.append(state_variables_mean)
    
 #   gw_state = cfe_purturb_instance_synth.get_value("DEEP_GW_TO_CHANNEL_FLUX")
  #  gw_statesssss = np.array(list(gw_state))
 #   gw_state_all.append(gw_state)
  #  gw_state_mean = np.mean(gw_statesssss)
   # gw_state_all_mean.append(gw_state_mean)
    

    Flow = observation_noisy[t]
    observations_all.append(Flow)
    
    enkf_instance.initialize()
    enkf_instance.set_value('Observations', Flow) 
    enkf_instance.set_value('Simulations', simulations)
    enkf_instance.set_value('State Variables', state_variables)
    enkf_instance.update()
    updated_state_sm = enkf_instance.get_value("state_estimates")
    updated_state_all.append(updated_state_sm)
    updated_state_sm = updated_state_sm[0]
    
    enkf_instance.finalize()
    #enkf_instance.initialize()
    

#    enkf_instance.set_value('Observations', Flow) 
#    enkf_instance.set_value('Simulations', simulations)
 #   enkf_instance.set_value('State Variables', gw_state)
  #  enkf_instance.update()
  #  updated_state_gw = enkf_instance.get_value("state_estimates")
   # updated_gw_state_all.append(updated_state_gw)
    #updated_state_gw = updated_state_gw[0]
    
   # enkf_instance.finalize()

#    cfe_instance.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip/1000)
 #   cfe_instance.set_value('water_potential_evaporation_flux', pet[t])
  #  cfe_instance.set_value('SOIL_CONCEPTUAL_STORAGE', updated_state_sm)
 #   cfe_instance.set_value('DEEP_GW_TO_CHANNEL_FLUX', updated_state_gw)
  #  cfe_instance.update()
   # simulation_updated = cfe_instance.get_value("land_surface_water__runoff_volume_flux")
  #  updated_simulations_all.append(simulation_updated)
    
    list_of_states_sm = [updated_state_sm] * 1000
    cfe_purturb_instance_synth.set_value('SOIL_CONCEPTUAL_STORAGE', list_of_states_sm)
    
    
#    list_of_states_gw = [updated_state_gw] * 1000
  #  cfe_purturb_instance_synth.set_value('DEEP_GW_TO_CHANNEL_FLUX', list_of_states_gw)
    
cfe_purturb_instance_synth.finalize()
enkf_instance.finalize()



In [ ]:
###############Calculate quantiles

open_quantile_5 = np.percentile(open_loop_simulations, 5, axis=1)
open_quantile_95 = np.percentile(open_loop_simulations, 95, axis=1)

da_quantile_5 = np.percentile(da_simulations_all, 5, axis=1)
da_quantile_95 = np.percentile(da_simulations_all, 95, axis=1)


open_quantile_15 = np.percentile(open_loop_simulations, 15, axis=1)
open_quantile_85 = np.percentile(open_loop_simulations, 85, axis=1)

da_quantile_15 = np.percentile(da_simulations_all, 15, axis=1)
da_quantile_85 = np.percentile(da_simulations_all, 85, axis=1)

In [ ]:
plt.close()
plt.plot(da_simulations_all_mean, color='blue', label='EnKF')
plt.plot(observation_noisy, color='black', label='Observation')
plt.plot(open_loop_simulations_mean, color='red', label='no DA')

plt.xlabel('Date')
plt.ylabel('Q')
plt.legend()
plt.plot()
#import matplotlib
#matplotlib.use('Agg')
#Save the plot as an image file
#plt.savefig('mean_Q.tif', backend='Agg')

In [ ]:
plt.close()
plt.plot(open_quantile_95, color='red', label='no DA Q95', linestyle='--')
plt.plot(open_quantile_5, color='red', label='no DA Q5', linestyle='--')
plt.plot(da_quantile_5, color='blue', label='EnKF Q5', linestyle='-.')
plt.plot(da_quantile_95, color='blue', label='EnKF Q95', linestyle='-.')
#plt.plot(open_quantile_85, color='red', label='Open Loop Q85', linestyle='--')
#plt.plot(open_quantile_15[60:100], color='red', label='Open Loop Q15', linestyle='--')
#plt.plot(da_quantile_15[60:100], color='blue', label='DA Loop Q15', linestyle='-.')
#plt.plot(da_quantile_85[60:100], color='blue', label='DA Loop Q85', linestyle='-.')
plt.plot(observation_noisy, color='black', label='Observation')




plt.xlabel('Date')
plt.ylabel('Q')
plt.legend()
plt.plot()
#import matplotlib
#matplotlib.use('Agg')
# Save the plot as an image file
#plt.savefig('synthetic_loop.tif', backend='Agg')

In [ ]:
from hydrostats.metrics import kge_2009 as kge

kge_score_DA = kge(da_simulations_all_mean, observation_noisy)
kge_score_OL = kge(open_loop_simulations_mean, observation_noisy)
print("Kling-Gupta Efficiency for DA:", kge_score_DA)
print("Kling-Gupta Efficiency for OL:", kge_score_OL)

In [ ]:
from hydrostats.metrics import nse

nse_score_DA = nse(da_simulations_all_mean, observation_noisy)
nse_score_OL = nse(open_loop_simulations_mean, observation_noisy)
print("NSE for DA:", nse_score_DA)
print("NSE for OL:", nse_score_OL)

In [ ]:
from hydrostats.metrics import rmse

rmse_score_DA = rmse(da_simulations_all_mean, observation_noisy)
rmse_score_OL = rmse(open_loop_simulations_mean, observation_noisy)
print("RMSE for DA:", rmse_score_DA)
print("RMSE for OL:", rmse_score_OL)

In [ ]:
plt.close()
plt.plot(state_variables_all_mean[2:], color='crimson', label='EnKF')
plt.plot(state_sm_all[2:], color='pink', label='Observation')
plt.plot(ostate_variables_all_mean[2:], color='blue', label='Open')
plt.show()
#plt.plot(updated_state_all[1:])

In [ ]:
np.save("syn_open_loop_simulations", open_loop_simulations)

In [ ]:
np.save("syn_da_simulations_all", da_simulations_all)


In [ ]:
np.save("syn_da_simulations_all_mean", da_simulations_all_mean)


In [ ]:
np.save("syn_observation_noisy", observation_noisy)


In [ ]:
np.save("syn_open_loop_simulations_mean", open_loop_simulations_mean)


In [ ]:
ostate_variables_all_mean